# Training Mistral 7B Instruction v2.0 using DPO: 

Follow this blog for reference: https://medium.com/@mauryaanoop3/dpo-fine-tuning-for-enhanced-language-model-performance-466fec349a5e

In [ ]:
!git config --global credential.helper store

# Install the huggingface_hub library
!pip install huggingface_hub -q

In [2]:
# Import the notebook_login method
from huggingface_hub import notebook_login

# Log in interactively
notebook_login()

In [3]:
!pip install datasets -q --no-cache

In [4]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("DhruvParth/Mistral-7B-Instruct-v2.0-PairRM-DPO-Dataset")

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

## 1. Environment Setup and Library Installation:

In [6]:
# Install necessary libraries
!pip install -q datasets trl bitsandbytes sentencepiece
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git

In [7]:
# Importing packages
import os
import gc
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
from trl import DPOTrainer
import bitsandbytes as bnb

2024-08-04 06:33:53.399195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-04 06:33:53.399295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-04 06:33:53.512023: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


This section installs the necessary libraries:
- trl: Provides the DPO training functionalities.
- bitsandbytes: Enables 4-bit quantization for memory efficiency.
- sentencepiece: For tokenization with SentencePiece models.
- transformers: The core library for working with pre-trained models.
- peft: Offers Parameter-Efficient Fine-Tuning (PEFT) techniques, specifically LoRA.

## 2. Model and Tokenizer Initialization:

In [8]:
# Define model names and tokens
peft_model_name = "mistralai/Mistral-7B-Instruct-v0.2" # The model obtained after the SFT step
new_model = "Mistral-7B-Instruct-v0.2-DPO-v0.1" #the name of the DPO trained model

# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained(peft_model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

We have already downloaded the dataset: 

In [10]:
ds.get("train")

Dataset({
    features: ['prompt_id', 'prompt', 'chosen', 'rejected', 'all_generated_resopnses', 'all_rm_scores'],
    num_rows: 50
})

## 3. LoRA Configuration and Model Loading:

In [11]:
# LoRA configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'v_proj', 'q_proj', 'dense']
)

# Load the base model with BitsAndBytes configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_name,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    is_trainable=True,
)

model.config.use_cache = False
model.load_adapter(peft_model_name, adapter_name="training2")
model.load_adapter(peft_model_name, adapter_name="reference")

ValueError: Can't find 'adapter_config.json' at 'mistralai/Mistral-7B-Instruct-v0.2'

Here, we configure LoRA and load the base model:

- The peft_config defines the LoRA parameters, a PEFT technique that significantly reduces the number of trainable parameters, making the fine-tuning process more efficient.
- The bnb_config configures BitsAndBytes for 4-bit quantization, further reducing memory usage.
- We load the pre-trained model using AutoPeftModelForCausalLM, applying the specified LoRA and quantization configurations.